# VOLTANDO UMA PASTA PARA TRAZ

In [ ]:
import os
os.chdir("../")

# PIPELINE EXTRAÇÃO

In [ ]:
from modules.youtube.extract.ytbpy import channel, playlist, utils, video
from modules import db_manager
from datetime import datetime
from jinja2 import Template
import pandas as pd
import yt_dlp
import time
import os

path_sql = "repository\\querys\\operation\\insert\\video_historic.sql"
with open(path_sql, "r", encoding="utf-8") as file:
    sql_query_insert = file.read()
   
with open("repository\\querys\\operation\\read\\video.sql", "r", encoding="utf-8") as file:
    sql_query_read = file.read()
    df_videos = db_manager.read_df(sql_query_read)

In [ ]:
for idx, row in df_videos.iterrows(): # .iloc[11096:]
    time.sleep(1)
    print("Processing video: {}/{}".format(idx + 1, len(df_videos)))
    video_info = video.get_video_info(url_or_id=row["url"], timeout=10)
    # cut = df_videos[df_videos.id_video.isin([ID_VIDEO])]
    # if cut.empty:
    
    ID_PLAYLIST     = row["id_playlist"]    if "id_playlist" in row else None
    TIME_MOVIE      = video_info["duration_seconds"] if "duration_seconds" in video_info else -1
    PUBLISHED_DATE  = video_info["published_date"] if "published_date" in video_info else "" 
    
    sql_template            = Template(sql_query_insert)
    rendered_sql            = sql_template.render(
        ID_MOVIE            = row["id_movie"],
        TITLE               = video_info["title"].replace("'", "").strip()                                                      if "title" in video_info else "",
        DESCRIPTION         = video_info["description"].replace("'", "").strip()                                                if "description" in video_info else None,
        VIEWS               = video_info["views_count"]                                                                         if "views_count" in video_info else None,
        LIKES               = video_info["likes_count"]                                                                         if "likes_count" in video_info else None,
        LIVE                = video_info["is_live"]                                                                             if "is_live" in video_info else None,
        KEYWORDS            = "'"+"', '".join([row.replace("'", "").strip() for row in video_info["keywords"]])+"'"             if "keywords" in video_info else None,
        AVAILABLE_COUNTRIES = "'"+"', '".join([row.replace("'", "").strip() for row in video_info["available_countries"]])+"'"   if "available_countries" in video_info else None,
        CATEGORY            = video_info["category"]                                                                            if "category" in video_info else None,
        FAMILY_FRIENDLY     = video_info["family_friendly"]                                                                     if "family_friendly" in video_info else None,
        DT_UPLOAD           = datetime.fromisoformat(video_info["upload_date_text"]).date().isoformat()                         if "upload_date_text" in video_info else None, 
        DT_PUBLISH          = datetime.fromisoformat(video_info["publish_date_text"]).date().isoformat()                        if "publish_date_text" in video_info else None,
        DT_START            = datetime.now().strftime("%Y-%m-%d"),
        DT_END              = '1900-01-01'
    ).replace("'None'", "NULL").replace("None", "NULL")\
     .replace("'False'", "FALSE").replace("False", "FALSE")\
     .replace("'True'", "TRUE").replace("True", "TRUE")
    
    try:
        db_manager.execute_raw(rendered_sql)
        print(f"✓ Video historic inserido: {video_info['title']}")
    except Exception as e:
        print(f"Error inserting video data: {e}")
        print(f"SQL: {rendered_sql}")
        db_manager.connection.rollback()
    

Processing video: 21812/92045
✓ Video historic inserido: OG.TOPSON BRINGING CLINKZ TO META - DOTA 2 7.24 GAMEPLAY
Processing video: 21813/92045
✓ Video historic inserido: NIGMA.MIRACLE CLINKZ WITH 19 KILLS - DOTA 2 7.23F GAMEPLAY
Processing video: 21814/92045
✓ Video historic inserido: NIGMA.MIRACLE CLINKZ - 22 MINS GG - DOTA 2 7.23F GAMEPLAY
Processing video: 21815/92045
✓ Video historic inserido: NIGMA.MIRACLE TRIES EVERYTHING TO WIN USING CLINKZ - DOTA 2 7.23E GAMEPLAY
Processing video: 21816/92045
✓ Video historic inserido: NIGMA.MIRACLE CLINKZ 14-0 KDA - DOTA 2 7.23E GAMEPLAY
Processing video: 21817/92045
✓ Video historic inserido: NIGMA.MIRACLE CLINKZ WITH 22 KILLS - DOTA 2 7.23E GAMEPLAY
Processing video: 21818/92045
✓ Video historic inserido: TNC.GABBI TRYING CLINKZ FOR THE FIRST TIME AFTER OUTLANDERS - DOTA 2 7.23 GAMEPLAY
Processing video: 21819/92045
✓ Video historic inserido: Liquid.Miracle Clinkz with 22 KILLS in 7.20 | Dota Gameplay
Processing video: 21820/92045
✓ Video h